In [1]:

!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=5959644 sha256=d2f85c008b5f4d4d3788b4f64e0d28a989a7d4a49fb3a9523e2e1404eee1912f
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from llama_cpp import Llama

model_path = "/content/drive/My Drive/Qwen2-500M-Instruct-Q4_K_M.gguf"
llm = Llama(model_path=model_path)

output = llm("Q: What is the capital of France? A:")
print("output is :" ,output)



llama_model_loader: loaded meta data with 25 key-value pairs and 290 tensors from /content/drive/My Drive/Qwen2-500M-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = Qwen2-0.5B-Instruct
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 896
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 4864
llama_model_loader: - kv   6:                 qwen2.attention.head_count u32              = 14
llama_model_loader: - kv   7:              qwe

output is : {'id': 'cmpl-4efbf8a2-b005-4aaa-8e72-8e51da317b75', 'object': 'text_completion', 'created': 1742703829, 'model': '/content/drive/My Drive/Qwen2-500M-Instruct-Q4_K_M.gguf', 'choices': [{'text': ' Luxembourg\nAnswer: No, the capital of France is Paris. Luxembourg is the', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 11, 'completion_tokens': 16, 'total_tokens': 27}}


In [8]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
from datasets import load_dataset
from llama_cpp import Llama
import time
from openai import OpenAI

svamp = load_dataset("ChilleD/SVAMP")

#model_path = "/content/drive/My Drive/Qwen2-500M-Instruct-Q4_K_M.gguf"
#model_path = "/content/drive/My Drive/Qwen2-500M-Instruct-Q8_0.gguf"
#model_path = "/content/drive/My Drive/Qwen2-500M-Instruct-f32.gguf"
#model_path = "/content/drive/My Drive/Qwen2-500M-Instruct-f32.gguf"
#model_path = "/content/drive/My Drive/Qwen2-1.5B.Q2_K.gguf"
model_path = "/content/drive/My Drive/Qwen2-1.5B.Q8_0.gguf"
llm = Llama(model_path=model_path)

client = OpenAI(api_key="not uploading because of security issues")


correct, total = 0, 20

start_time = time.time()
for example in svamp['test'].select(range(total)):
    question_text = example['Body'] + " " + example['Question']
    correct_answer = str(example['Answer'])
    prompt = f"<|im_start|>user\n{question_text}\nLet's think step by step.<|im_end|>\n<|im_start|>assistant\n"

    tokens = llm.tokenize(prompt.encode("utf-8"))
    print(f"Tokenized input ({len(tokens)} tokens): {tokens[:20]} ...")

    response = llm(
        prompt,
        max_tokens=256,
        temperature=0,
        stop=["<|im_end|>"]
    )

    full_response = response['choices'][0]['text'].strip()

    verification_prompt = f"""Question: {question_text} Model Response: {full_response} Correct Answer: {correct_answer} Based on the logical analysis of the Model Response and the Correct Answer provided, is the model's response correct? Reply ONLY with a single word: TRUE or FALSE."""

    chatgpt_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": verification_prompt}],
        temperature=0
    )

    verification_result = chatgpt_response.choices[0].message.content.strip().upper()

    print(f"Question: {question_text}")
    print(f"Model Response:\n{full_response}")
    print(f"ChatGPT Verification: {verification_result}")

    if verification_result == "TRUE":
        correct += 1

end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f}s")


accuracy = correct / total
print(f"Accuracy on SVAMP dataset: {accuracy:.2%}")


llama_model_loader: loaded meta data with 21 key-value pairs and 338 tensors from /content/drive/My Drive/Qwen2-1.5B.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 28
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 131072
llama_model_loader: - kv   4:                     qwen2.embedding_length u32              = 1536
llama_model_loader: - kv   5:                  qwen2.feed_forward_length u32              = 8960
llama_model_loader: - kv   6:                 qwen2.attention.head_count u32              = 12
llama_model_loader: - kv   7:              qwen2.attention.head_coun

Tokenized input (90 tokens): [27, 91, 318, 4906, 91, 29, 872, 198, 66188, 374, 4558, 1588, 323, 1429, 9898, 525, 84526, 311, 44939, 5837] ...


llama_perf_context_print:        load time =    6420.10 ms
llama_perf_context_print: prompt eval time =    6419.81 ms /    77 tokens (   83.37 ms per token,    11.99 tokens per second)
llama_perf_context_print:        eval time =   26957.96 ms /   111 runs   (  242.86 ms per token,     4.12 tokens per second)
llama_perf_context_print:       total time =   33591.46 ms /   188 tokens


Question: Winter is almost here and most animals are migrating to warmer countries. There are 41 bird families living near the mountain. If 35 bird families flew away to asia and 62 bird families flew away to africa How many more bird families flew away to africa than those that flew away to asia?
Model Response:
Based on the information provided, we can set up the following equation:

35 bird families flew away to Asia
62 bird families flew away to Africa

We want to find out how many more bird families flew away to Africa than those that flew away to Asia. So we can subtract the number of bird families that flew away to Asia from the number of bird families that flew away to Africa:

62 - 35 = 27

So, 27 more bird families flew away to Africa than those that flew away to Asia.
ChatGPT Verification: TRUE
Tokenized input (64 tokens): [27, 91, 318, 4906, 91, 29, 872, 198, 20291, 7256, 9226, 220, 22, 6623, 18170, 323, 220, 16, 17, 8251] ...


Llama.generate: 3 prefix-match hit, remaining 48 prompt tokens to eval
llama_perf_context_print:        load time =    6420.10 ms
llama_perf_context_print: prompt eval time =    3017.45 ms /    48 tokens (   62.86 ms per token,    15.91 tokens per second)
llama_perf_context_print:        eval time =   16616.97 ms /    70 runs   (  237.39 ms per token,     4.21 tokens per second)
llama_perf_context_print:       total time =   19764.29 ms /   118 tokens


...

Question: A farmer had 177 tomatoes and 12 potatoes in his garden. If he picked 53 tomatoes How many tomatoes and potatoes does he have left?
Model Response:
Based on the information provided, we can set up the following equation to find out how many tomatoes and potatoes the farmer has left:

177 tomatoes - 53 tomatoes = 12 potatoes

So the farmer has 12 potatoes left. To find out how many tomatoes he has left, we can subtract the number of tomatoes he picked from the total number of tomatoes:

177 tomatoes - 53 tomatoes = 124 tomatoes

Therefore, the farmer has 124 tomatoes and 12 potatoes left.
ChatGPT Verification: FALSE
Tokenized input (67 tokens): [27, 91, 318, 4906, 91, 29, 872, 198, 97010, 51207, 16486, 18568, 13, 1260, 5558, 220, 21, 21, 16486, 18568] ...


Llama.generate: 3 prefix-match hit, remaining 51 prompt tokens to eval
llama_perf_context_print:        load time =    6420.10 ms
llama_perf_context_print: prompt eval time =    3304.04 ms /    51 tokens (   64.79 ms per token,    15.44 tokens per second)
llama_perf_context_print:        eval time =    4032.12 ms /    18 runs   (  224.01 ms per token,     4.46 tokens per second)
llama_perf_context_print:       total time =    7368.79 ms /    69 tokens


Question: Danny collects bottle caps. He lost 66 bottle caps at the park. Now he has 25 bottle caps in his collection. How many bottle caps did danny have at first?
Model Response:
Danny had 25 + 66 = 91 bottle caps at first.
ChatGPT Verification: TRUE
Tokenized input (63 tokens): [27, 91, 318, 4906, 91, 29, 872, 198, 32631, 3949, 220, 19, 14298, 304, 279, 6556, 11, 220, 20, 14298] ...


Llama.generate: 3 prefix-match hit, remaining 47 prompt tokens to eval
llama_perf_context_print:        load time =    6420.10 ms
llama_perf_context_print: prompt eval time =    4186.99 ms /    47 tokens (   89.08 ms per token,    11.23 tokens per second)
llama_perf_context_print:        eval time =   10397.62 ms /    44 runs   (  236.31 ms per token,     4.23 tokens per second)
llama_perf_context_print:       total time =   14663.14 ms /    91 tokens


Question: Jack received 4 emails in the morning, 5 emails in the afternoon and 8 emails in the evening. How many emails did Jack receive in the afternoon and evening?
Model Response:
Based on the information provided, Jack received 5 emails in the afternoon and 8 emails in the evening. So in total, he received 5 + 8 = 13 emails in the afternoon and evening.
ChatGPT Verification: TRUE
Time taken: 506.07s
Accuracy on SVAMP dataset: 50.00%
